# Test plan

## Use ADM for evaluating the sensitivity of the ik-rvea

			

## Setting up the iADM
Change capitalized variable names as needed

In [15]:
#Change these

# List of problems to be solved
PROBLEMS = ['DTLZ1' , 'DTLZ2', 'DTLZ4', 'DTLZ5', 'DTLZ7']    

# Objective functions
NFs = [4,6,8]
# Path to results directory
RESDIR = './results'
# Path to matlab files

# Number of reference points/runs
RUNS=15

In [16]:
import os
import pickle
# Make certain we have neccessery directories
if not os.path.exists(RESDIR):
    os.makedirs(RESDIR)

def fn(fn):
    """Helper to access result files"""
    return os.path.join(RESDIR,fn)


def preferences(problem, nf, it, resdir="deprecated"):
    preferences = pickle.load(open(fn('preferences.dmp'), 'r'))
    pref = preferences[(int(nf), problem)][int(it)]
    pareto_points = pickle.load(open(fn('PO.dmp'), 'r'))
    aspPO = pareto_points[(int(nf), problem, pref)]
    return (pref, aspPO)


%config IPCompleter.greedy=True

## Generate intial preferences

Initial preferences are calculated and corresponding PO solutions are found  before MATLAB tests. Here we have total of `max_runs` different preference items for each objective

This is to speed up the calcualtions esp due slow initialization of JVM

These should be already available, if not, they must calculated

After these are calculated, run matlab


# Collect results

Generate  results in IK-RVEA directory using script
```sh
run.sh logfile.log
```

Then collect them

In [20]:
import pandas as pd
import glob
import os
res_list = []
for f in glob.glob(os.path.join(fn('.'),'*.csv')):
    res_list.append(pd.read_csv(f,header=0,index_col=None))
df=pd.concat(res_list,ignore_index=True)
POs=pickle.load(open(fn(("PO.dmp"))))                    

In [18]:
pinst = [k for k in df.keys() if k not in [u'pref_id', u'refs', u'PO']]
problems={}
for p in pinst:
    problems[p]=list(pd.unique(df[p]))


In [29]:
from scipy import spatial,stats
import numpy as np

# We are getting some (unncessary warnings)
import warnings
warnings.filterwarnings('ignore')

res_columns=['problem','nf','Evals','Update','TrainD', 'median', 'deviation','min','median i']                        

rows_list=[]
for problem in problems["problem"]:
    for k in problems["nf"]:
        try:
            max_runs = int(max(df[(df.nf==k)&(df.problem==problem)]["pref_id"]))+1
            aspPOs=[]
            for i in range(max_runs):
                ref = preferences(problem,k,i,RESDIR)[0]
                aspPOs.append(POs[k,problem,tuple(ref)])
            for fe in problems["Evals"]:
                for train in problems["TrainD"]:
                    for update in problems["Update"]:
                        dist=[]
                        iters=[]
                        pinst = (df.nf==k)&(df.problem==problem)&(df.Evals==fe)&(df.TrainD==train)&(df.Update==update)
                        for pref_id, res in df[pinst].iterrows():
                            PO = eval(res.PO)[-1]
                            aspPO = aspPOs[int(res.pref_id)]
                            dist.append(spatial.distance.euclidean(PO,aspPO))
                            iters.append(len(eval(res.refs)))
                        row = {'problem':problem,
                               'nf':k,
                               'Evals':fe,
                               'TrainD':train,
                               'Update':update,
                               'median':np.median(dist),
                               'deviation':np.std(dist),
                               'min': np.min(dist),
                               'median i': np.median(iters)}
                        rows_list.append(row)
        except ValueError:
            # Check missing data here
            continue

res_df=pd.DataFrame(rows_list)
res_df=res_df[res_columns]


# View results
Filter results by the column, as given below

In [30]:
from IPython.display import display, HTML
df1=res_df[(res_df.problem=="DTLZ2")&(res_df.Evals==300)]
df2=res_df[(res_df.problem=="DTLZ2")&(res_df.Evals==500)]
display(HTML(df1.to_html()))
display(HTML(df2.to_html()))

,problem,nf,Evals,Update,TrainD,median,deviation,min,median i
0,DTLZ2,6,300.0,1.0,2.0,0.954182,0.280534,0.493139,4.0
1,DTLZ2,6,300.0,5.0,2.0,1.012539,0.288247,0.715199,4.0
2,DTLZ2,6,300.0,10.0,2.0,0.940946,0.225760,0.402997,4.0
3,DTLZ2,6,300.0,1.0,6.0,0.966666,0.263623,0.437698,4.0
4,DTLZ2,6,300.0,5.0,6.0,1.037996,0.291826,0.758556,4.0
5,DTLZ2,6,300.0,10.0,6.0,1.020759,0.197942,0.725275,4.0
6,DTLZ2,6,300.0,1.0,10.0,0.992143,0.216706,0.614826,4.0
7,DTLZ2,6,300.0,5.0,10.0,0.925571,0.220077,0.585563,4.0
8,DTLZ2,6,300.0,10.0,10.0,1.051117,0.283900,0.573917,4.0
18,DTLZ2,4,300.0,1.0,2.0,1.185581,0.254896,0.718086,4.0


,problem,nf,Evals,Update,TrainD,median,deviation,min,median i
9,DTLZ2,6,500.0,1.0,2.0,0.892195,0.244037,0.735829,4.0
10,DTLZ2,6,500.0,5.0,2.0,0.948868,0.228614,0.593004,4.0
11,DTLZ2,6,500.0,10.0,2.0,1.008420,0.225376,0.471946,4.0
12,DTLZ2,6,500.0,1.0,6.0,1.006870,0.237331,0.649870,4.0
13,DTLZ2,6,500.0,5.0,6.0,1.000352,0.229058,0.572927,4.0
14,DTLZ2,6,500.0,10.0,6.0,0.994624,0.169683,0.775606,4.0
15,DTLZ2,6,500.0,1.0,10.0,1.123663,0.306246,0.598878,4.0
16,DTLZ2,6,500.0,5.0,10.0,1.031969,0.200292,0.762607,4.0
17,DTLZ2,6,500.0,10.0,10.0,0.931091,0.227316,0.549059,4.0
27,DTLZ2,4,500.0,1.0,2.0,1.260756,0.230009,0.750767,4.0


## Latex tables
Filter similarly as above

In [26]:
from IPython.core.display import display, HTML
for problem in PROBLEMS:
    print res_df[res_df.problem==problem].to_latex(index=False,float_format = lambda x:"%11.4f" % x)

\begin{tabular}{lrrrrrrrr}
\toprule
problem &  nf &       Evals &      Update &      TrainD &      median &   deviation &         min &    median i \\
\midrule
  DTLZ1 &   6 &    300.0000 &      1.0000 &      2.0000 &    181.3460 &     25.5253 &    129.0959 &    299.0000 \\
  DTLZ1 &   4 &    300.0000 &      1.0000 &      2.0000 &    348.3175 &     26.2351 &    290.7548 &    143.0000 \\
  DTLZ1 &   4 &    300.0000 &      5.0000 &      2.0000 &    336.3410 &     33.0998 &    245.4610 &    174.0000 \\
  DTLZ1 &   4 &    300.0000 &     10.0000 &      2.0000 &    331.1459 &     60.1515 &    185.9983 &    178.0000 \\
  DTLZ1 &   4 &    300.0000 &      1.0000 &      6.0000 &    341.2836 &     26.4143 &    292.5094 &    171.0000 \\
  DTLZ1 &   4 &    300.0000 &      5.0000 &      6.0000 &    338.4362 &     28.9999 &    297.7388 &    123.0000 \\
  DTLZ1 &   4 &    300.0000 &     10.0000 &      6.0000 &    332.2578 &     21.0372 &    293.8695 &    165.0000 \\
  DTLZ1 &   4 &    300.0000 &      